In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [7]:
def doi_url(row):
    query = ''
    if not pd.isna(row['Item DOI']):
        query = row['Item DOI']
        return 1,query
    elif not pd.isna(row['URL']):
        query = row['URL']
        return 2,query

In [8]:
df = pd.read_csv('wide_search_no_abstract.csv') #csv com abstracts faltantes

In [9]:
# Iterate over the DataFrame rows
total = len(df)
i = 1
for index, row in df.iterrows():
    if pd.isna(row['Abstract']):
        # Extract the DOI for each row
        op,query = doi_url(row)

        if op == 1:
            doi = query
            article_url = f'https://link.springer.com/{doi}'
        elif op == 2:
            article_url = query

        # Send an HTTP GET request to the article URL
        response = requests.get(article_url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the article page
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find the abstract section using the HTML structure you provided
            abstract_section = soup.find('section', {'data-title': 'Abstract'})

            if abstract_section:
                # Extract the text content of the abstract
                abstract_text = abstract_section.find('div', {'class': 'c-article-section__content'}).get_text()
                # Update the 'Abstract Note' column in the original DataFrame 'df'
                df.loc[index, 'Abstract'] = abstract_text
            else:
                df.loc[index, 'Abstract'] = "Abstract not found."
        else:
            df.loc[index, 'Abstract'] = "Failed to retrieve data."
            print(f"Failed to retrieve data for DOI {doi}. Status code: {response.status_code}")
        print(f'{i} out of {total} concluded')
        i = i+1

1 out of 9749 concluded
2 out of 9749 concluded
3 out of 9749 concluded
4 out of 9749 concluded
5 out of 9749 concluded
6 out of 9749 concluded
7 out of 9749 concluded
8 out of 9749 concluded
9 out of 9749 concluded
10 out of 9749 concluded
11 out of 9749 concluded
12 out of 9749 concluded
13 out of 9749 concluded
14 out of 9749 concluded
15 out of 9749 concluded
16 out of 9749 concluded
17 out of 9749 concluded
18 out of 9749 concluded
19 out of 9749 concluded
20 out of 9749 concluded
21 out of 9749 concluded
22 out of 9749 concluded
23 out of 9749 concluded
24 out of 9749 concluded
25 out of 9749 concluded
26 out of 9749 concluded
27 out of 9749 concluded
28 out of 9749 concluded
29 out of 9749 concluded
30 out of 9749 concluded
31 out of 9749 concluded
32 out of 9749 concluded
33 out of 9749 concluded
34 out of 9749 concluded
35 out of 9749 concluded
36 out of 9749 concluded
37 out of 9749 concluded
38 out of 9749 concluded
39 out of 9749 concluded
40 out of 9749 concluded
41 out of

In [10]:
df.to_csv('springer-wide-raw-search-with-abstract.csv') #novo csv com abstracts preenchidos